# [Misplaced Priorities - Part 2](https://xpuzzlecorner.substack.com/p/misplaced-priorities-part-2)
## January 12, 2025

## Problem

_Given that these queued passengers did such a poor job of getting into the correct order the first time, you don’t trust them them to get it right this time. So instead you have each of them display their ticket while you move them around. You can move people by selecting two people and swapping their locations.  This is the only movement operation you’re allowed.  If we assume there are 10 people in line and their order is completely random, how many swaps will it take to get them ordered correctly? How about for N people?_

## Solution

We can use a some of the work we did in [Part 1](https://xpuzzlecorner.substack.com/p/misplaced-priorities) to help us solve this problem. We know that the only people who need to be swapped are the ones who are out of order. Everyone that is out of order we can treat as a groups and we know this group is a derangement of k elements (where k is the size of the group). We also already know to compute the probability of there being k people in the incorrect spot so that will help us as well. What we're left with is trying to figure out how many swaps will be required to re-arrange the derangement subset of the group.

So how do we do that? Let's first examine the structure of a derangement (or really any permuation at all). Recall that every permutation can be [written as a product of disjoint cycles](https://en.wikipedia.org/wiki/Permutation#:~:text=line%20notation%2C%20etc.\).-,A%20permutation,is%20called%20a%20k%2Dcycle.%20\(See%20%C2%A7%C2%A0Cycle%20notation%20below.\),-A%20fixed%20point). For example, the permutation (12)(34) on 4 elements is composed of two cyles each of size 2. The permutation (1234) is a single cycle of size 4.

Any derangement must be composed of cycles with length greater than 1. This is because if a cycle has length 1, then that element is in the correct spot. 

But how do we correct each of these cycles. Let's consider a cycle of length 2. We can correct this cycle with a single swap. For example, the cycle (12) can be corrected by swapping 1 and 2. How about a cycle with length 3? We can correct this cycle with 2 swaps. For example, the cycle (123) can be corrected by swapping 1 and 2 and then swapping 1 and 3. So it appears that we can correct a cycle of length k with k-1 swaps. To verify this, we consider how the cycle could have been made (since a swap operation is completely reversable). if we consider a element that's mapped to itself as a cycle of length 1, then every swap operation you do will increase the length of a cycle by 1. And since you can't arbitrarily merge 2 cycles with length >2 into a single cycle, you can't reduce the length of a cycle by more than 1. So we can correct a cycle of length k with k-1 swaps.

So no we just need to compute the probability distribution of various cycle lengths associated with a derangement. We can do this by considering the number of ways to partition a set of size k into cycles of length greater than 1.

In [3]:
def truncate(lst, n):
    ans = []
    total = 0
    for i in lst:
        if i >= 2:
            ans.append(i)
            total += i
    if total == n:
        return ans
    else:
        return []

def GeneratePartitions(n):
    p = [0] * (n + 1)  # An array to store a partition
    k = 0  # Index of last element in a partition
    p[k] = n  # Initialize first partition as number itself

    # This loop first prints current partition,
    # then generates next partition. The loop
    # stops when the current partition has all 1s
    while True:
        # Generate next partition
        partition = truncate(p, n)
        if partition:
            yield partition

        # Find the rightmost non-one value in p[].
        # Also, update the rem_val so that we know
        # how much value can be accommodated
        rem_val = 0
        while k >= 0 and p[k] == 1:
            rem_val += p[k]
            k -= 1

        # if k < 0, all the values are 1 so
        # there are no more partitions
        if k < 0:
            return

        # Decrease the p[k] found above
        # and adjust the rem_val
        p[k] -= 1
        rem_val += 1

        # If rem_val is more, then the sorted
        # order is violated. Divide rem_val in
        # different values of size p[k] and copy
        # these values at different positions after p[k]
        while rem_val > p[k]:
            p[k + 1] = p[k]
            rem_val = rem_val - p[k]
            k += 1

        # Copy rem_val to next position
        # and increment position
        p[k + 1] = rem_val
        k += 1


In [46]:
for partition in GeneratePartitions(5):
    print(partition)

[5]
[3, 2]


Returning back to our discussion about the distribution of cycles and their lengths, we can break can consider the probability of each possible derangement size for our set of size n.  Part 1 showed us how to compute this.  From here, we need to generate the different cycles sizes that can be achieved for a derangement of that size (the partitions of k where each partition is $\geq 2$). Then finally, we need to compute the probability of that particular cycle structure given a derangement of that size.
4
More formally, let $S$ be the number of swaps needed to correct a random permutation. Let $P_k$ be the set of integer partition of $k$ where every partition $p$ is ≥2 and each partition p = {c1, c2, …} where $c_i$ is a part of the partition $p$. Let $K$ be be the event where $k$ of the $n$ elements of a random permutation are fixed. Finally, $S(p)$ is the number of swaps needed to correct a derangement of $k$ elements and is given by

$$
S(p) = \sum_{c_i \in p} c_i - 1
$$

$$
E[S] = \sum_{k=0}^n \left[ \sum_{p \in P_k} S(p) P(p|K) \right] P(K)
$$

We computed $ P(\text{k fixed elements}) $ in part 1.

$$
P(p|K) = \frac{k!}{\displaystyle \prod_{p \in P_k} p! \cdot \prod_{s \in \text{\{\# partitions of size s\}}} s!}
$$

In [42]:
import math
from collections import Counter

# def D(k):
#     # Compute $D_k = n! \sum_{i=0}^n \frac{(-1)^i}{i!}$
#     return math.factorial(k) * sum((-1)**i / math.factorial(i) for i in range(k + 1))

def D(k):
    if k == 0:
        return 1
    if k == 1:
        return 0

    prev = 1  # D_0
    curr = 0  # D_1

    for i in range(2, k + 1):
        next_val = (i - 1) * (prev + curr)
        prev, curr = curr, next_val

    return curr


def numNcycles(n):
    return math.factorial(n - 1)

def numWaysToFormGroups(groups):
    total = sum(groups)
    group_size_counts = Counter(groups)
    numerator = math.factorial(total)
    denominator1 = math.prod([math.factorial(x) for x in groups])
    indistinguishable = math.prod([math.factorial(x) for x in group_size_counts.values()])
    return numerator // (denominator1 * indistinguishable)

def numSwaps(groups):
    return sum([g-1 for g in groups])

In [30]:
ns = range(5,11)

for n in ns:
    total = 0
    for partition in GeneratePartitions(n):
        v = numWaysToFormGroups(partition) * math.prod([numNcycles(x) for x in partition])
        # print(partition, v)
        total += v
    print(f"n: {n}, D(n): {D(n)}, num: {total}")
    assert total == D(n)

n: 5, D(n): 44, num: 44
n: 6, D(n): 265, num: 265
n: 7, D(n): 1854, num: 1854
n: 8, D(n): 14833, num: 14833
n: 9, D(n): 133496, num: 133496
n: 10, D(n): 1334961, num: 1334961


In [45]:
n = 10

weighted_averages = []
for k in range(0,n+1):
    d_k = D(k)
    probability_k = (1 / (math.factorial(n-k)*math.factorial(k))) * d_k
    if k == 0:
        avg_num_swaps = 0
    elif k == 1:
        avg_num_swaps = 0
    else:
        partitions = GeneratePartitions(k)
        avg_num_swaps = 0
        for partition in partitions:
            num_partition = numWaysToFormGroups(partition) * math.prod([numNcycles(x) for x in partition])
            avg_num_swaps += numSwaps(partition) * (num_partition/d_k)
    
    weighted_averages.append(probability_k * avg_num_swaps)
    # print(f"k: {k}, D(k): {d_k}, probability: {probability_k}, avg_num_swaps: {avg_num_swaps}")

print(sum(weighted_averages))

7.071031746031747


## Proof that the expected number of cylcles in a random permutation is the harmonic number

After attempting this proof a bit myself, I get a little stumped and didn't want to spend a lot of time on it, so I decided to instead look it up. Somewhat embarrassingly, most of what I found I couldn't really understand until I found this video from [Professor Roman Vershynin](https://www.youtube.com/watch?app=desktop&v=7MJKMD65IVA&ab_channel=RomanVershynin).

Since every permutation is composed of disjoint cycles, Let's write the permutation as $p = \{c_1, c_2, \cdots , c_k \}$ where $c$ is a set containing all the elements in a cycle $c = \{ x_1, x_2, \cdots , x_l \}$. Let $C(X_i) = |c_i|$ be the size of the cycle that contains $x_i$. 

With this, we can write create the random variable $Y_i = \frac{1}{C(X_i)}$. We can then write the total number of cycles as $Y = \sum_{i=1}^k Y_i$. We want to compute $E[Y]$.

$$
E[Y] = E\left[ \sum_{i=1}^k Y_i \right] = \sum_{i=1}^k E[Y_i]
$$

Since there's nothing special about any of the elements in the cycle, we can write $E[Y_i] = E[Y_j]$ for all $i,j$. So we can write

$$
E[Y] = n E[Y_i] = n \sum_{i=1}^n \frac{1}{n} \cdot P(C(X_i) = n) = \sum_{i=1}^n P(C(X_i) = n)
$$

So we just need to compute the probability that a random element is in a cycle of size $n$. Let's start by consider a cycle of length 1, For some arbitrary element, there's one option for where that element can be mapped so the probability is $\frac{1}{n}$. For a cycle of length 2, we need to ensure that the element is not mapped to itself. There are $n-1$ options for where the first element can be mapped and then element 2 needs to map back to element 1. So the probability is $\frac{n-1}{n} \cdot \frac{1}{n-1} = \frac{1}{n}$. For a cycle of length 3, we need to ensure that the element is not mapped to itself and that the second element is not mapped to the first element. There are $n-1$ options for where the first element can be mapped, $n-2$ options for where the second element can be mapped, and then the third element needs to map back to the first element. So the probability is $\frac{n-1}{n} \cdot \frac{n-2}{n-1} \cdot \frac{1}{n-2} = \frac{1}{n}$. We can continue this pattern for all cycles of length $n$ and we see that the probability that a random element is in a cycle of length $n$ is $\frac{1}{n}$. So we can write

$$
E[Y] = \sum_{i=1}^n \frac{1}{n}
$$

This is the definition of the harmonic number. So the expected number of cycles in a random permutation is the harmonic number n where n is the size of the permutation.